In [36]:
import pandas as pd
import numpy as np
import os

DATA_INTERIM   = "../data/interim"
DATA_PROCESSED = "../data/processed"


In [37]:
# ---- Health datasets ----
health_life     = pd.read_csv(f"{DATA_INTERIM}/life_expectancy_clean.csv")
health_child     = pd.read_csv(f"{DATA_INTERIM}/child_mortality_clean.csv")
health_infant    = pd.read_csv(f"{DATA_INTERIM}/infant_mortality_clean.csv")
health_ncd       = pd.read_csv(f"{DATA_INTERIM}/mortality_ncds_clean.csv")
health_obesity   = pd.read_csv(f"{DATA_INTERIM}/obesity_prevalence_clean.csv")
health_water     = pd.read_csv(f"{DATA_INTERIM}/water_access_full_clean.csv")

# ---- Infrastructure datasets ----
infra_fuels      = pd.read_csv(f"{DATA_INTERIM}/clean_fuels_cooking.csv")
infra_elec       = pd.read_csv(f"{DATA_INTERIM}/electricity_generation_clean.csv")
infra_renew      = pd.read_csv(f"{DATA_INTERIM}/renewables_share_clean.csv")
infra_energy     = pd.read_csv(f"{DATA_INTERIM}/energy_use_per_capita_clean.csv")
infra_oda        = pd.read_csv(f"{DATA_INTERIM}/oda_infrastructure_clean.csv")
infra_disasters  = pd.read_csv(f"{DATA_INTERIM}/disaster_deaths_clean.csv")

print("Datasets loaded successfully.")


Datasets loaded successfully.


In [38]:
from functools import reduce

health_datasets = [
    health_life,
    health_child,
    health_infant,
    health_ncd,
    health_obesity,
    health_water
]

health_df = reduce(lambda left, right: pd.merge(
    left, right, on=["country", "iso_code", "year"], how="outer"), health_datasets)

print("Health merged shape:", health_df.shape)
health_df.head()


Health merged shape: (19618, 15)


,country,iso_code,year,life_expectancy,child_mortality_rate,infant_mortality_rate,ncd_mortality,obesity_rate,water_safely_managed,water_basic,water_limited,water_unimproved,water_surface,water_access_total,water_inadequate_total
0,Afghanistan,AFG,1950,28.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1951,28.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1952,29.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1953,29.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1954,29.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
infra_datasets = [
    infra_fuels,
    infra_elec,
    infra_renew,
    infra_energy,
    infra_oda,
    infra_disasters
]

infra_df = reduce(lambda left, right: pd.merge(
    left, right, on=["country", "iso_code", "year"], how="outer"), infra_datasets)

print("Infrastructure merged shape:", infra_df.shape)
infra_df.head()


Infrastructure merged shape: (12535, 9)


,country,iso_code,year,clean_cooking_access,electricity_gen_per_capita,renewables_share,energy_use_per_capita,oda_infrastructure,disaster_deaths
0,Afghanistan,AFG,1900,NaN,NaN,NaN,NaN,NaN,0.0
1,Afghanistan,AFG,1910,NaN,NaN,NaN,NaN,NaN,0.0
2,Afghanistan,AFG,1920,NaN,NaN,NaN,NaN,NaN,0.0
3,Afghanistan,AFG,1930,NaN,NaN,NaN,NaN,NaN,0.0
4,Afghanistan,AFG,1940,NaN,NaN,NaN,NaN,NaN,0.0


In [40]:
merged = pd.merge(
    health_df,
    infra_df,
    on=["country", "iso_code", "year"],
    how="outer"
)

print("Merged total shape:", merged.shape)
merged.head()


Merged total shape: (20793, 21)


,country,iso_code,year,life_expectancy,child_mortality_rate,infant_mortality_rate,ncd_mortality,obesity_rate,water_safely_managed,water_basic,...,water_unimproved,water_surface,water_access_total,water_inadequate_total,clean_cooking_access,electricity_gen_per_capita,renewables_share,energy_use_per_capita,oda_infrastructure,disaster_deaths
0,Afghanistan,AFG,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,Afghanistan,AFG,1910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,Afghanistan,AFG,1920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,Afghanistan,AFG,1930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,Afghanistan,AFG,1940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [41]:
merged_trimmed = merged[(merged["year"] >= 2000) & (merged["year"] <= 2021)].copy()

print("Rows before:", len(merged))
print("Rows after trimming:", len(merged_trimmed))
merged_trimmed.head()


Rows before: 20793
Rows after trimming: 5278


,country,iso_code,year,life_expectancy,child_mortality_rate,infant_mortality_rate,ncd_mortality,obesity_rate,water_safely_managed,water_basic,...,water_unimproved,water_surface,water_access_total,water_inadequate_total,clean_cooking_access,electricity_gen_per_capita,renewables_share,energy_use_per_capita,oda_infrastructure,disaster_deaths
55,Afghanistan,AFG,2000,55.00,13.17,110.1,43.2,3.69,11.093327,16.348530,...,43.856777,25.402164,27.441857,72.558144,6.20,23.844612,64.583336,262.68048,440000.0,447.5
56,Afghanistan,AFG,2001,55.51,12.74,107.0,43.5,4.01,11.105221,16.368359,...,43.843445,25.383093,27.473580,72.526421,7.10,34.016453,72.463770,179.89190,420000.0,NaN
57,Afghanistan,AFG,2002,56.23,12.31,103.8,43.1,4.34,12.007733,17.667130,...,42.260395,24.457567,29.674863,70.325139,8.30,33.211520,78.873245,155.69543,27640000.0,NaN
58,Afghanistan,AFG,2003,57.17,11.87,100.6,42.5,4.70,12.909922,18.965668,...,40.677280,23.533058,31.875590,68.124410,9.35,40.029820,69.230770,174.55792,126350000.0,NaN
59,Afghanistan,AFG,2004,57.81,11.42,97.2,42.3,5.08,13.818684,20.275747,...,39.086002,22.598950,34.094431,65.905569,10.80,33.530476,70.886080,157.75752,533680000.0,NaN


In [42]:
missing_summary = merged_trimmed.isna().sum().to_frame("Missing_Count")
missing_summary["Missing_%"] = (missing_summary["Missing_Count"] / len(merged_trimmed)) * 100

missing_summary.sort_values("Missing_%", ascending=False)


,Missing_Count,Missing_%
disaster_deaths,4609,87.324744
oda_infrastructure,2074,39.295188
water_safely_managed,2050,38.840470
water_basic,2050,38.840470
ncd_mortality,1186,22.470633
clean_cooking_access,1054,19.969685
obesity_rate,878,16.635089
infant_mortality_rate,856,16.218264
child_mortality_rate,845,16.009852
water_surface,693,13.129973


In [43]:
infra_cols = [
    "clean_cooking_access",
    "electricity_gen_per_capita",
    "renewables_share",
    "energy_use_per_capita",
    "oda_infrastructure",
    "disaster_deaths"
]

merged_final = merged_trimmed.dropna(subset=infra_cols, how="all")

print("Final rows after infra-row filtering:", len(merged_final))


Final rows after infra-row filtering: 5037


In [45]:
if "disaster_deaths" in merged_final.columns:
    merged_final = merged_final.drop(columns=["disaster_deaths"])
    print("Removed disaster_deaths column.")
else:
    print("disaster_deaths column not found.")


Removed disaster_deaths column.


In [46]:
missing_summary_after = merged_final.isna().sum().to_frame("Missing_Count")
missing_summary_after["Missing_%"] = missing_summary_after["Missing_Count"] / len(merged_final) * 100
missing_summary_after.sort_values("Missing_%", ascending=False)


,Missing_Count,Missing_%
water_safely_managed,1900,37.720866
water_basic,1900,37.720866
oda_infrastructure,1833,36.390709
ncd_mortality,945,18.761167
clean_cooking_access,813,16.140560
obesity_rate,638,12.666270
infant_mortality_rate,625,12.408179
child_mortality_rate,614,12.189796
water_surface,570,11.316260
water_limited,486,9.648600


In [51]:
# List of columns to interpolate
cols_to_interpolate = [
    'water_safely_managed', 'water_basic', 'oda_infrastructure',
    'ncd_mortality', 'clean_cooking_access', 'obesity_rate',
    'infant_mortality_rate', 'child_mortality_rate', 'water_surface',
    'water_limited', 'water_unimproved', 'renewables_share',
    'electricity_gen_per_capita', 'water_access_total',
    'water_inadequate_total', 'energy_use_per_capita'
]

merged_final_interp = merged_final.copy()

for col in cols_to_interpolate:
    merged_final_interp[col] = (
        merged_final_interp
        .groupby("country")[col]
        .transform(lambda group: group.interpolate(limit=5))
    )

print("Interpolation complete.")


Interpolation complete.


In [53]:
merged_final_interp.isna().sum().sort_values(ascending=False)


water_safely_managed          1888
water_basic                   1888
oda_infrastructure            1752
ncd_mortality                  945
clean_cooking_access           813
obesity_rate                   638
infant_mortality_rate          625
child_mortality_rate           609
water_surface                  458
water_limited                  404
water_unimproved               404
renewables_share               334
electricity_gen_per_capita     312
energy_use_per_capita          255
water_inadequate_total         209
water_access_total             209
life_expectancy                 33
year                             0
country                          0
iso_code                         0
dtype: int64

In [54]:
missing_after = merged_final_interp.isna().sum().to_frame("Missing_Count")
missing_after["Missing_%"] = missing_after["Missing_Count"] / len(merged_final_interp) * 100
missing_after.sort_values("Missing_%", ascending=False)


,Missing_Count,Missing_%
water_safely_managed,1888,37.482629
water_basic,1888,37.482629
oda_infrastructure,1752,34.782609
ncd_mortality,945,18.761167
clean_cooking_access,813,16.140560
obesity_rate,638,12.666270
infant_mortality_rate,625,12.408179
child_mortality_rate,609,12.090530
water_surface,458,9.092714
water_limited,404,8.020647


In [55]:
# 1. Keep only 2000–2020
merged_cut = merged[(merged["year"] >= 2000) & (merged["year"] <= 2020)].copy()

# 2. Remove aggregated regions (no ISO code)
merged_cut = merged_cut[merged_cut["iso_code"].notna()]

# 3. Drop very incomplete variables
cols_drop = [
    "water_safely_managed", 
    "water_basic",
    "water_limited",
    "water_unimproved",
    "water_surface",
    "oda_infrastructure"   # too incomplete
]

merged_cut = merged_cut.drop(columns=[c for c in cols_drop if c in merged_cut.columns])

# 4. Interpolation for small gaps
cols_to_interp = [
    "life_expectancy", "child_mortality_rate", "infant_mortality_rate",
    "ncd_mortality", "obesity_rate",
    "clean_cooking_access", "renewables_share",
    "electricity_gen_per_capita", "energy_use_per_capita"
]

for col in cols_to_interp:
    merged_cut[col] = (
        merged_cut.groupby("country")[col]
        .transform(lambda s: s.interpolate(limit=3))
    )

# 5. Forward fill + backward fill
merged_cut = (
    merged_cut.groupby("country")
    .apply(lambda g: g.ffill().bfill())
    .reset_index(drop=True)
)

# 6. Drop countries with >50% missing values
missing_threshold = merged_cut.isna().mean(axis=1)
merged_final = merged_cut[missing_threshold < 0.5].copy()

print("Final shape:", merged_final.shape)


Final shape: (4725, 15)


C:\Users\wpmpo\AppData\Local\Temp\ipykernel_16376\211368191.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.ffill().bfill())


In [56]:
# Check missing values AFTER cleaning
missing_summary = merged_final.isna().sum().to_frame("Missing_Count")
missing_summary["Missing_%"] = (
    merged_final.isna().mean() * 100
).round(2)

missing_summary.sort_values("Missing_%", ascending=False)


,Missing_Count,Missing_%
ncd_mortality,819,17.33
clean_cooking_access,693,14.67
obesity_rate,525,11.11
infant_mortality_rate,504,10.67
child_mortality_rate,483,10.22
disaster_deaths,294,6.22
renewables_share,231,4.89
electricity_gen_per_capita,210,4.44
energy_use_per_capita,168,3.56
water_inadequate_total,42,0.89


In [57]:
os.makedirs("../data/processed", exist_ok=True)

# Save the final cleaned dataset
merged_final.to_csv("../data/processed/merged_final_clean.csv", index=False)

print("Saved cleaned dataset as: data/processed/merged_final_clean.csv")
print("Final dataset shape:", merged_final.shape)

Saved cleaned dataset as: data/processed/merged_final_clean.csv
Final dataset shape: (4725, 15)
